# AIS Heatmaps

In [1]:
# base libraries
import numpy as np
import pandas as pd
import datetime
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium as fm
from folium.plugins import HeatMap
import branca
from folium.features import DivIcon

In [2]:
# set variable from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
shipping_filename = config['DEFAULT']['shipping_filename']

### Data Import

In [3]:
dtype_dic = {'MMSI':int,'dt':'str', 'lat':'float', 'long':'float','SOG':'float', 'ROT':'float', 
             'Type':'str', 'gross_tonnage':'float','vessel_name':'str', 'ETA':'str', 'POC_LOCODE':'str',
             'last_port_LOCODE':'str', 'next_port_LOCODE':'str', 'status':'str','voyage_id':'float','tripid':int,
            'in_hazmat':'str','out_hazmat':'str'}
parse_dates = ['dt', 'ETA']

shipping_data = pd.read_csv(processing_path + shipping_filename,header = 0,delimiter = ',',dtype = dtype_dic, parse_dates=parse_dates)

### Heatmaps

AIS data can be used to explore the voyages of individual ships. The daily points taken from within an area can be combined to produce a heatmap of ship movements within a port, which can be used to indicate shipping lanes, port berths and port loading over time.

In [4]:
# create list of tuples to plot 
shipping_data['points'] = shipping_data[['lat','long']].values.tolist()
# split into interesting dates for maps
sd_date1 = shipping_data[shipping_data['dt'].dt.date == datetime.date(2016,12,12)].copy()
sd_date2 = shipping_data[shipping_data['dt'].dt.date == datetime.date(2016,12,19)].copy()
sd_date3 = shipping_data[shipping_data['dt'].dt.date == datetime.date(2016,12,25)].copy()
sd_date4 = shipping_data[shipping_data['dt'].dt.date == datetime.date(2017,1,1)].copy()

In [5]:
map_location = [51.9506,1.295]
map_zoom = 13
map_width = '48%'
map_height = '48%'
map_position='absolute'

# create the maps
map0 = fm.Map(width=map_width, height=map_height, position=map_position, left='0%', top = '0%', location = map_location, zoom_start=13)
map1 = fm.Map(width=map_width, height=map_height, position=map_position, left='50%', top= '0%', location = map_location, zoom_start=13)
map2 = fm.Map(width=map_width, height=map_height, position=map_position, left='0%', top = '50%', location = map_location, zoom_start=13)
map3 = fm.Map(width=map_width, height=map_height, position=map_position, left='50%', top = '50%', location = map_location, zoom_start=13)

# annotate with the date
fm.map.Marker([51.964,1.298],icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),
                                           html='<div style="font-size: 16pt">12th December</div>',)
             ).add_to(map0)
fm.map.Marker([51.964,1.298],icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),
                                           html='<div style="font-size: 16pt">19th December</div>',)
             ).add_to(map1)
fm.map.Marker([51.964,1.298],icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),
                                           html='<div style="font-size: 16pt">25th December</div>',)
             ).add_to(map2)
fm.map.Marker([51.964,1.298],icon=DivIcon(icon_size=(150,36),icon_anchor=(0,0),
                                           html='<div style="font-size: 16pt">1st January</div>',)
             ).add_to(map3)

HeatMap(sd_date1['points'], radius = 4, blur = 3).add_to(map0)
HeatMap(sd_date2['points'], radius = 4, blur = 3).add_to(map1)
HeatMap(sd_date3['points'], radius = 4, blur = 3).add_to(map2)
HeatMap(sd_date4['points'], radius = 4, blur = 3).add_to(map3)

The heatmaps below highlight notable features relating to Felixstowe (the busiest container port in the UK), including:

* The emergence of “hot” regions indicating unique berths within the northern edge of the port, most notably on the 12th of December and the 1st of January
* Evidence that not all ships that enter Felixstowe go onto dock at the port. Some ships turn east and head towards the international port of Harwich whilst others head north west onto the river Orwell and onto inland destinations
* Port loading on Christmas day is considerably lower than on other days

In [7]:
f1 = branca.element.Figure()
f1.add_child(map0)
f1.add_child(map1)
f1.add_child(map2)
f1.add_child(map3)